# Libraries

In [ ]:
import numpy as np
import pandas as pd
from time import time

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pandas import set_option
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import seaborn as sns; sns.set() 

import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics
import re
#from sklearn.preprocessing import Imputer
from numpy import random
import seaborn as sb
import matplotlib.pyplot as plt

from itertools import cycle
plt.style.use('ggplot')

from scipy.stats import percentileofscore
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
import os
from sklearn.ensemble import ExtraTreesClassifier
#from yellowbrick.style.palettes import LINE_COLOR

In [ ]:
#https://www.javaer101.com/article/2986770.html
#https://www.kite.com/python/answers/how-to-compute-the-confidence-interval-of-a-sample-statistic-in-python

import numpy as np
import scipy
from scipy.stats import sem
from sklearn.metrics import roc_auc_score

def ROC_CI_y_true_y_pred(y_true,y_pred):
    roc_auc = auc(y_true, y_pred)
    #print("Original ROC area: {:0.3f}".format(roc_auc))

    n_bootstraps = 1000
    rng_seed = 42  # control reproducibility
    bootstrapped_scores = []

    rng = np.random.RandomState(rng_seed)
    for i in range(n_bootstraps):
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.randint(0, len(y_pred), len(y_pred))
        if len(np.unique(y_true[indices])) < 2:
            # We need at least one positive and one negative sample for ROC AUC
            # to be defined: reject the sample
            continue

        score = auc(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)
        #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))
    sample = np.array(bootstrapped_scores)
    confidence_level = 0.95
    degrees_freedom = sample.size - 1
    sample_mean = np.mean(sample)
    sample_standard_error = scipy.stats.sem(sample)

    confidence_interval = scipy.stats.t.interval(confidence_level, degrees_freedom, sample_mean, sample_standard_error)
    min_ = confidence_interval[0]
    max_ = confidence_interval[1]
    return roc_auc, min_, max_

# Functions

In [ ]:
def score_ci(
    y_true,
    y_pred,
    score_fun,
    n_bootstraps=2000,
    confidence_level=0.95,
    seed=None,
    reject_one_class_samples=True,
):
    """
    Compute confidence interval for given score function based on labels and predictions using bootstrapping.
    :param y_true: 1D list or array of labels.
    :param y_pred: 1D list or array of predictions corresponding to elements in y_true.
    :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
    :param n_bootstraps: The number of bootstraps. (default: 2000)
    :param confidence_level: Confidence level for computing confidence interval. (default: 0.95)
    :param seed: Random seed for reproducibility. (default: None)
    :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
    need at least one positive and one negative sample. (default: True)
    :return: Score evaluated on labels and predictions, lower confidence interval, upper confidence interval, array of
    bootstrapped scores.
    """

    assert len(y_true) == len(y_pred)

    score = score_fun(y_true, y_pred)
    _, ci_lower, ci_upper, scores = score_stat_ci(
        y_true=y_true,
        y_preds=y_pred,
        score_fun=score_fun,
        n_bootstraps=n_bootstraps,
        confidence_level=confidence_level,
        seed=seed,
        reject_one_class_samples=reject_one_class_samples,
    )

    return score, ci_lower, ci_upper, scores


def score_stat_ci(
    y_true,
    y_preds,
    score_fun,
    stat_fun=np.mean,
    n_bootstraps=2000,
    confidence_level=0.95,
    seed=None,
    reject_one_class_samples=True,
):
    """
    Compute confidence interval for given statistic of a score function based on labels and predictions using
    bootstrapping.
    :param y_true: 1D list or array of labels.
    :param y_preds: A list of lists or 2D array of predictions corresponding to elements in y_true.
    :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
    :param stat_fun: Statistic for which confidence interval is computed. (e.g. np.mean)
    :param n_bootstraps: The number of bootstraps. (default: 2000)
    :param confidence_level: Confidence level for computing confidence interval. (default: 0.95)
    :param seed: Random seed for reproducibility. (default: None)
    :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
    need at least one positive and one negative sample. (default: True)
    :return: Mean score statistic evaluated on labels and predictions, lower confidence interval, upper confidence
    interval, array of bootstrapped scores.
    """

    y_true = np.array(y_true)
    y_preds = np.atleast_2d(y_preds)
    assert all(len(y_true) == len(y) for y in y_preds)

    np.random.seed(seed)
    scores = []
    for i in range(n_bootstraps):
        readers = np.random.randint(0, len(y_preds), len(y_preds))
        indices = np.random.randint(0, len(y_true), len(y_true))
        if reject_one_class_samples and len(np.unique(y_true[indices])) < 2:
            continue
        reader_scores = []
        for r in readers:
            reader_scores.append(score_fun(y_true[indices], y_preds[r][indices]))
        scores.append(stat_fun(reader_scores))

    mean_score = np.mean(scores)
    sorted_scores = np.array(sorted(scores))
    alpha = (1.0 - confidence_level) / 2.0
    ci_lower = sorted_scores[int(round(alpha * len(sorted_scores)))]
    ci_upper = sorted_scores[int(round((1.0 - alpha) * len(sorted_scores)))]
    return mean_score, ci_lower, ci_upper, scores


def pvalue(
    y_true,
    y_pred1,
    y_pred2,
    score_fun,
    n_bootstraps=2000,
    two_tailed=True,
    seed=None,
    reject_one_class_samples=True,
):
    """
    Compute p-value for hypothesis that score function for model I predictions is higher than for model II predictions
    using bootstrapping.
    :param y_true: 1D list or array of labels.
    :param y_pred1: 1D list or array of predictions for model I corresponding to elements in y_true.
    :param y_pred2: 1D list or array of predictions for model II corresponding to elements in y_true.
    :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
    :param n_bootstraps: The number of bootstraps. (default: 2000)
    :param two_tailed: Whether to use two-tailed test. (default: True)
    :param seed: Random seed for reproducibility. (default: None)
    :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
    need at least one positive and one negative sample. (default: True)
    :return: Computed p-value, array of bootstrapped differences of scores.
    """

    assert len(y_true) == len(y_pred1)
    assert len(y_true) == len(y_pred2)

    return pvalue_stat(
        y_true=y_true,
        y_preds1=y_pred1,
        y_preds2=y_pred2,
        score_fun=score_fun,
        n_bootstraps=n_bootstraps,
        two_tailed=two_tailed,
        seed=seed,
        reject_one_class_samples=reject_one_class_samples,
    )


def pvalue_stat(
    y_true,
    y_preds1,
    y_preds2,
    score_fun,
    stat_fun=np.mean,
    n_bootstraps=2000,
    two_tailed=True,
    seed=None,
    reject_one_class_samples=True,
):
    """
    Compute p-value for hypothesis that given statistic of score function for model I predictions is higher than for
    model II predictions using bootstrapping.
    :param y_true: 1D list or array of labels.
    :param y_preds1: A list of lists or 2D array of predictions for model I corresponding to elements in y_true.
    :param y_preds2: A list of lists or 2D array of predictions for model II corresponding to elements in y_true.
    :param score_fun: Score function for which confidence interval is computed. (e.g. sklearn.metrics.accuracy_score)
    :param stat_fun: Statistic for which p-value is computed. (e.g. np.mean)
    :param n_bootstraps: The number of bootstraps. (default: 2000)
    :param two_tailed: Whether to use two-tailed test. (default: True)
    :param seed: Random seed for reproducibility. (default: None)
    :param reject_one_class_samples: Whether to reject bootstrapped samples with only one label. For scores like AUC we
    need at least one positive and one negative sample. (default: True)
    :return: Computed p-value, array of bootstrapped differences of scores.
    """

    y_true = np.array(y_true)
    y_preds1 = np.atleast_2d(y_preds1)
    y_preds2 = np.atleast_2d(y_preds2)
    assert all(len(y_true) == len(y) for y in y_preds1)
    assert all(len(y_true) == len(y) for y in y_preds2)

    np.random.seed(seed)
    z = []
    for i in range(n_bootstraps):
        readers1 = np.random.randint(0, len(y_preds1), len(y_preds1))
        readers2 = np.random.randint(0, len(y_preds2), len(y_preds2))
        indices = np.random.randint(0, len(y_true), len(y_true))
        if reject_one_class_samples and len(np.unique(y_true[indices])) < 2:
            continue
        reader_scores = []
        for r in readers1:
            reader_scores.append(score_fun(y_true[indices], y_preds1[r][indices]))
        score1 = stat_fun(reader_scores)
        reader_scores = []
        for r in readers2:
            reader_scores.append(score_fun(y_true[indices], y_preds2[r][indices]))
        score2 = stat_fun(reader_scores)
        z.append(score1 - score2)

    p = percentileofscore(z, 0.0, kind="weak") / 100.0
    if two_tailed:
        p *= 2.0
    return p, z

In [ ]:
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
#https://github.com/BioAITeam/Steganalysis/blob/main/MV.py
#https://github.com/BioAITeam/Steganalysis/blob/main/GBRAS-Net.ipynb
#https://github.com/mateuszbuda/ml-stat-util/blob/master/stat_util.py
#https://pubs.rsna.org/doi/pdf/10.1148/radiol.2020202944

def ROC_curves(y_test,y_score,name, path_img_base = './images'):
    if not os.path.exists(path_img_base):
        os.makedirs(path_img_base)
    n_classes = y_test.shape[1]
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

        
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    #roc_auc, min_, max_ = ROC_CI_y_true_y_pred(tpr["micro"],fpr["micro"])
    #print(roc_auc, min_, max_) 
    
    # Compute macro-average ROC curve and ROC area
    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    # Finally average it and compute AUC
    mean_tpr /= n_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    

    # Plot ROC curve
    plt.figure(figsize=(10, 10))
    classes_names=["DB","SS","SW","A","I","B"]
    for i in range(n_classes):
        score, ci_lower, ci_upper, scores = score_ci(y_test[:, i], y_score[:, i], 
                                                       score_fun=roc_auc_score,
                                                       seed=42)
        
        text="ROC curve with AUC of {:.3f} (95% CI: {:.3f}, {:.3f}). "+"Class "+classes_names[i]

        plt.plot(fpr[i], tpr[i], 
                 label=text.format(score, ci_lower, ci_upper))
    
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (AUC = {0:0.3f})'.format(roc_auc["micro"]),
             color='aqua', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (AUC = {0:0.3f})'.format(roc_auc["macro"]),
             color='darkorange', linestyle=':', linewidth=4)

    plt.plot([0, 1], [0, 1], linestyle=":", c=LINE_COLOR) 
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.tick_params(axis='x', which='major', labelsize=16) 
    plt.tick_params(axis='y', which='major', labelsize=16)
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title("ROC Curves for {}".format(name), fontsize=16)
    plt.legend(loc="lower right", frameon=True, fontsize=15)
    plt.savefig(path_img_base+'/ROC_CI_graphs_per_class_'+name+'.svg', format='svg') 
    plt.savefig(path_img_base+'/ROC_CI_graphs_per_class_'+name+'.pdf', format='pdf') 
    plt.show()

# Dataset

## Shoham

In [ ]:
model_ = ExtraTreesClassifier(n_estimators=112, max_depth=31, min_samples_split=5, random_state=28000001)

In [ ]:
PATH_DATA = '../Databases/ShohamDB.csv' 

In [ ]:
## Loading Data

# Velocity, Viscosity, Density, Surface Tension, Angle and Diameter
dataset = pd.DataFrame(pd.read_csv(PATH_DATA), columns=['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID', 'Flow Pattern']) 

# Summarize the Dataset 
print("shape of initial data =",dataset.shape) 
# Class Distribution 
print(dataset.groupby('Flow Pattern').size()) 
# Leaving only the best training variables
dataset = dataset.drop(['VisG', 'VisL','DenG', 'ST', 'DenL'], axis=1) #Delete this variables
print("shape of selected data =",dataset.shape) 

print(dataset.head()) 

# Split-out validation dataset 
array = dataset.values 
X = array[:,0:4] #Data or features 
y = array[:,4]   #Label or classes 

validation_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=validation_size, random_state=510) 

print("\ntrain data shape =",X_train.shape) 
print("train labels shape =",y_train.shape) 
print("test data shape =",X_test.shape) 
print("test labels shape =",y_test.shape)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = model_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_score=accuracy_score(y_test,y_pred) 
print('accuracy_score: {0:.4f}'.format(acc_score))

In [ ]:
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = y
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
print(integer_encoded.shape)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
print(onehot_encoded.shape)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)


validation_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, onehot_encoded, test_size=validation_size, random_state=510) 
print("\ntrain data shape =",X_train.shape) 
print("train labels shape =",y_train.shape) 
print("test data shape =",X_test.shape) 
print("test labels shape =",y_test.shape)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = model_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_score=accuracy_score(y_test,y_pred) 
print('accuracy_score: {0:.4f}'.format(acc_score))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = model_
clf.fit(X_train,y_train)
y_score = clf.predict_proba(X_test)
print(np.array(y_score).shape)

In [ ]:
y_test.shape

In [ ]:
np.array(y_score).shape

In [ ]:
t0=y_score[0][:,1]
t1=y_score[1][:,1]
t2=y_score[2][:,1]
t3=y_score[3][:,1]
t4=y_score[4][:,1]
t5=y_score[5][:,1]

In [ ]:
p=[t0,t1,t2,t3,t4,t5]
np.array(p).shape

In [ ]:
w=np.asmatrix(p).T
w.shape

In [ ]:
y_score=np.squeeze(np.asarray(w))
y_score.shape

In [ ]:
print(y_test)
print(y_score)

In [ ]:
print(y_test.shape,y_score.shape)

In [ ]:
#run twice
from yellowbrick.style.palettes import LINE_COLOR

name = "Extra Trees model using Shoham data"
path = "./images/Shoham"
ROC_curves(y_test,y_score,name,path_img_base=path)

In [ ]:
fig,ax = plt.subplots(nrows=2,ncols=2,figsize=(20,20))

In [ ]:
from yellowbrick.style.palettes import LINE_COLOR

name = "Extra Trees model using Shoham data"
path_img_base = "./images/Shoham"
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)
n_classes = y_test.shape[1]
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#roc_auc, min_, max_ = ROC_CI_y_true_y_pred(tpr["micro"],fpr["micro"])
#print(roc_auc, min_, max_) 

# Compute macro-average ROC curve and ROC area
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
# Finally average it and compute AUC
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])


# Plot ROC curve
#plt.figure(figsize=(10, 10))

classes_names=["DB","SS","SW","A","I","B"]
for i in range(n_classes):
    score, ci_lower, ci_upper, scores = score_ci(y_test[:, i], y_score[:, i], 
                                                   score_fun=roc_auc_score,
                                                   seed=42)

    text="ROC curve with AUC of {:.3f} (95% CI: {:.3f}, {:.3f}). "+"Class "+classes_names[i]

    ax[0,0].plot(fpr[i], tpr[i], 
             label=text.format(score, ci_lower, ci_upper))

ax[0,0].plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (AUC = {0:0.3f})'.format(roc_auc["micro"]),
         color='aqua', linestyle=':', linewidth=4)

ax[0,0].plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (AUC = {0:0.3f})'.format(roc_auc["macro"]),
         color='darkorange', linestyle=':', linewidth=4)

ax[0,0].tick_params(axis='x', which='major', labelsize=16) 
ax[0,0].tick_params(axis='y', which='major', labelsize=16)
ax[0,0].plot([0, 1], [0, 1], linestyle=":", c=LINE_COLOR) 
ax[0,0].set_xlim([0.0, 1.0])
ax[0,0].set_ylim([0.0, 1.0])
ax[0,0].set_xlabel('False Positive Rate', fontsize=17)
ax[0,0].set_ylabel('True Positive Rate', fontsize=17)
ax[0,0].set_title("ROC Curves for {}".format(name), fontsize=18)
ax[0,0].legend(loc="lower right", frameon=True, fontsize=15)


## 12DB

In [ ]:
#model_ = ExtraTreesClassifier(n_estimators=220, max_depth=None, min_samples_split=3, random_state=18) #6
#model_ = ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=3, random_state=17) #5
#model_ = ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=38) #3

### 6 classes

In [ ]:
model_ = ExtraTreesClassifier(n_estimators=220, max_depth=None, min_samples_split=3, random_state=18) #6

In [ ]:
# Load Dataset
PATH = '../Databases/12DB_6FP.csv'

Data = pd.read_csv(PATH, names=['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID', 'Flow Pattern'], header=0)
print('Data shape:', Data.shape)
Data.head()


In [ ]:
Data.describe()

In [ ]:
# Obtain the class distribution
Data['Flow Pattern'].value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
# Train, test split
features_list = ['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID']
Features = Data[features_list]
Labels = Data['Flow Pattern']

X_train, X_test, y_train, y_test = train_test_split(Features, Labels, test_size=0.2, stratify=Labels, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print('Train data shape:', X_train.shape)
print('Train labels shape:', y_train.shape)
print('Test data shape:', X_test.shape)
print('Test labels shape:', y_test.shape)

In [ ]:
model = model_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_score=accuracy_score(y_test,y_pred) 
print('accuracy_score: {0:.4f}'.format(acc_score))

In [ ]:
Labels = np.expand_dims(Labels, axis=1)
Labels.shape

In [ ]:
integer_encoded=Labels
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
print(onehot_encoded.shape)


validation_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(Features, onehot_encoded, test_size=validation_size, random_state=510) 
print("\ntrain data shape =",X_train.shape) 
print("train labels shape =",y_train.shape) 
print("test data shape =",X_test.shape) 
print("test labels shape =",y_test.shape)


In [ ]:
model = model_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_score=accuracy_score(y_test,y_pred) 
print('accuracy_score: {0:.4f}'.format(acc_score))

In [ ]:
clf = model_
clf.fit(X_train,y_train)
y_score = clf.predict_proba(X_test)
print(np.array(y_score).shape)

In [ ]:
t0=y_score[0][:,1]
t1=y_score[1][:,1]
t2=y_score[2][:,1]
t3=y_score[3][:,1]
t4=y_score[4][:,1]
t5=y_score[5][:,1]

p=[t0,t1,t2,t3,t4,t5]
np.array(p).shape

w=np.asmatrix(p).T
w.shape

y_score=np.squeeze(np.asarray(w))
y_score.shape

print(y_test)
print(y_score)

print(y_test.shape,y_score.shape)

In [ ]:
from yellowbrick.style.palettes import LINE_COLOR

name = "Extra Trees model using 12 DB with 6 classes"
path_img_base = "./images/Shoham"
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)
n_classes = y_test.shape[1]
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#roc_auc, min_, max_ = ROC_CI_y_true_y_pred(tpr["micro"],fpr["micro"])
#print(roc_auc, min_, max_) 

# Compute macro-average ROC curve and ROC area
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
# Finally average it and compute AUC
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])


# Plot ROC curve
#plt.figure(figsize=(10, 10))

classes_names=["DB","SS","SW","A","I","B"]
for i in range(n_classes):
    score, ci_lower, ci_upper, scores = score_ci(y_test[:, i], y_score[:, i], 
                                                   score_fun=roc_auc_score,
                                                   seed=42)

    text="ROC curve with AUC of {:.3f} (95% CI: {:.3f}, {:.3f}). "+"Class "+classes_names[i]

    ax[0,1].plot(fpr[i], tpr[i], 
             label=text.format(score, ci_lower, ci_upper))

ax[0,1].plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (AUC = {0:0.3f})'.format(roc_auc["micro"]),
         color='aqua', linestyle=':', linewidth=4)

ax[0,1].plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (AUC = {0:0.3f})'.format(roc_auc["macro"]),
         color='darkorange', linestyle=':', linewidth=4)

ax[0,1].tick_params(axis='x', which='major', labelsize=16) 
ax[0,1].tick_params(axis='y', which='major', labelsize=16)
ax[0,1].plot([0, 1], [0, 1], linestyle=":", c=LINE_COLOR) 
ax[0,1].set_xlim([0.0, 1.0])
ax[0,1].set_ylim([0.0, 1.0])
ax[0,1].set_xlabel('False Positive Rate', fontsize=17)
ax[0,1].set_ylabel('True Positive Rate', fontsize=17)
ax[0,1].set_title("ROC Curves for {}".format(name), fontsize=18)
ax[0,1].legend(loc="lower right", frameon=True, fontsize=15)


### 5 classes

In [ ]:
#model_ = ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=3, random_state=17)
model_ = ExtraTreesClassifier(n_estimators=200, max_depth=31, min_samples_split=3, random_state=125)

In [ ]:
# Load Dataset
PATH = '../Databases/12DB_5FP.csv'

Data = pd.read_csv(PATH, names=['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID', 'Flow Pattern'], header=0)
print('Data shape:', Data.shape)
Data.head()


In [ ]:
Data.describe()

In [ ]:
# Obtain the class distribution
Data['Flow Pattern'].value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
# Train, test split
features_list = ['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID']
Features = Data[features_list]
Labels = Data['Flow Pattern']

X_train, X_test, y_train, y_test = train_test_split(Features, Labels, test_size=0.2, stratify=Labels, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print('Train data shape:', X_train.shape)
print('Train labels shape:', y_train.shape)
print('Test data shape:', X_test.shape)
print('Test labels shape:', y_test.shape)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = model_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_score=accuracy_score(y_test,y_pred) 
print('accuracy_score: {0:.4f}'.format(acc_score))

In [ ]:
Labels = np.expand_dims(Labels, axis=1)
Labels.shape

In [ ]:
integer_encoded=Labels
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
print(onehot_encoded.shape)


validation_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(Features, onehot_encoded, test_size=validation_size, random_state=510) 
print("\ntrain data shape =",X_train.shape) 
print("train labels shape =",y_train.shape) 
print("test data shape =",X_test.shape) 
print("test labels shape =",y_test.shape)


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = model_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_score=accuracy_score(y_test,y_pred) 
print('accuracy_score: {0:.4f}'.format(acc_score))

In [ ]:
clf = model_
clf.fit(X_train,y_train)
y_score = clf.predict_proba(X_test)
print(np.array(y_score).shape)

In [ ]:
t0=y_score[0][:,1]
t1=y_score[1][:,1]
t2=y_score[2][:,1]
t3=y_score[3][:,1]
t4=y_score[4][:,1]

p=[t0,t1,t2,t3,t4]
np.array(p).shape

w=np.asmatrix(p).T
w.shape

y_score=np.squeeze(np.asarray(w))
y_score.shape

print(y_test)
print(y_score)

print(y_test.shape,y_score.shape)

In [ ]:
from yellowbrick.style.palettes import LINE_COLOR

name = "Extra Trees model using 12 DB with 5 classes"
path_img_base = "./images/Shoham"
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)
n_classes = y_test.shape[1]
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#roc_auc, min_, max_ = ROC_CI_y_true_y_pred(tpr["micro"],fpr["micro"])
#print(roc_auc, min_, max_) 

# Compute macro-average ROC curve and ROC area
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
# Finally average it and compute AUC
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])


# Plot ROC curve
#plt.figure(figsize=(10, 10))

classes_names=["DB","ST","A","I","B"]
for i in range(n_classes):
    score, ci_lower, ci_upper, scores = score_ci(y_test[:, i], y_score[:, i], 
                                                   score_fun=roc_auc_score,
                                                   seed=42)

    text="ROC curve with AUC of {:.3f} (95% CI: {:.3f}, {:.3f}). "+"Class "+classes_names[i]

    ax[1,0].plot(fpr[i], tpr[i], 
             label=text.format(score, ci_lower, ci_upper))

ax[1,0].plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (AUC = {0:0.3f})'.format(roc_auc["micro"]),
         color='aqua', linestyle=':', linewidth=4)

ax[1,0].plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (AUC = {0:0.3f})'.format(roc_auc["macro"]),
         color='darkorange', linestyle=':', linewidth=4)

ax[1,0].tick_params(axis='x', which='major', labelsize=16) 
ax[1,0].tick_params(axis='y', which='major', labelsize=16)
ax[1,0].plot([0, 1], [0, 1], linestyle=":", c=LINE_COLOR) 
ax[1,0].set_xlim([0.0, 1.0])
ax[1,0].set_ylim([0.0, 1.0])
ax[1,0].set_xlabel('False Positive Rate', fontsize=17)
ax[1,0].set_ylabel('True Positive Rate', fontsize=17)
ax[1,0].set_title("ROC Curves for {}".format(name), fontsize=18)
ax[1,0].legend(loc="lower right", frameon=True, fontsize=15)


### 3 classes

In [ ]:
model_ = ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=38)

In [ ]:
# Load Dataset
PATH = '../Databases/12DB_3FP.csv'

Data = pd.read_csv(PATH, names=['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID', 'Flow Pattern'], header=0)
print('Data shape:', Data.shape)
Data.head()


In [ ]:
Data.describe()

In [ ]:
# Obtain the class distribution
Data['Flow Pattern'].value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
# Train, test split
features_list = ['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID']
Features = Data[features_list]
Labels = Data['Flow Pattern']

X_train, X_test, y_train, y_test = train_test_split(Features, Labels, test_size=0.2, stratify=Labels, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print('Train data shape:', X_train.shape)
print('Train labels shape:', y_train.shape)
print('Test data shape:', X_test.shape)
print('Test labels shape:', y_test.shape)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = model_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_score=accuracy_score(y_test,y_pred) 
print('accuracy_score: {0:.4f}'.format(acc_score))

In [ ]:
Labels = np.expand_dims(Labels, axis=1)
Labels.shape

In [ ]:
integer_encoded=Labels
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
print(onehot_encoded.shape)


validation_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(Features, onehot_encoded, test_size=validation_size, random_state=510) 
print("\ntrain data shape =",X_train.shape) 
print("train labels shape =",y_train.shape) 
print("test data shape =",X_test.shape) 
print("test labels shape =",y_test.shape)


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = model_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc_score=accuracy_score(y_test,y_pred) 
print('accuracy_score: {0:.4f}'.format(acc_score))

In [ ]:
clf = model_
clf.fit(X_train,y_train)
y_score = clf.predict_proba(X_test)
print(np.array(y_score).shape)

In [ ]:
t0=y_score[0][:,1]
t1=y_score[1][:,1]
t2=y_score[2][:,1]

p=[t0,t1,t2]
np.array(p).shape

w=np.asmatrix(p).T
w.shape

y_score=np.squeeze(np.asarray(w))
y_score.shape

print(y_test)
print(y_score)

print(y_test.shape,y_score.shape)

In [ ]:
from yellowbrick.style.palettes import LINE_COLOR

name = "Extra Trees model using 12 DB with 3 classes"
path_img_base = "./images/Shoham"
if not os.path.exists(path_img_base):
    os.makedirs(path_img_base)
n_classes = y_test.shape[1]
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#roc_auc, min_, max_ = ROC_CI_y_true_y_pred(tpr["micro"],fpr["micro"])
#print(roc_auc, min_, max_) 

# Compute macro-average ROC curve and ROC area
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
# Finally average it and compute AUC
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])


# Plot ROC curve
#plt.figure(figsize=(10, 10))

classes_names=["Dispersed","Segregate","Intermittent"]
for i in range(n_classes):
    score, ci_lower, ci_upper, scores = score_ci(y_test[:, i], y_score[:, i], 
                                                   score_fun=roc_auc_score,
                                                   seed=42)

    text="ROC curve with AUC of {:.3f} (95% CI: {:.3f}, {:.3f}). "+"Class "+classes_names[i]

    ax[1,1].plot(fpr[i], tpr[i], 
             label=text.format(score, ci_lower, ci_upper))

ax[1,1].plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (AUC = {0:0.3f})'.format(roc_auc["micro"]),
         color='aqua', linestyle=':', linewidth=4)

ax[1,1].plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (AUC = {0:0.3f})'.format(roc_auc["macro"]),
         color='darkorange', linestyle=':', linewidth=4)

ax[1,1].tick_params(axis='x', which='major', labelsize=16) 
ax[1,1].tick_params(axis='y', which='major', labelsize=16)
ax[1,1].plot([0, 1], [0, 1], linestyle=":", c=LINE_COLOR) 
ax[1,1].set_xlim([0.0, 1.0])
ax[1,1].set_ylim([0.0, 1.0])
ax[1,1].set_xlabel('False Positive Rate', fontsize=17)
ax[1,1].set_ylabel('True Positive Rate', fontsize=17)
ax[1,1].set_title("ROC Curves for {}".format(name), fontsize=18)
ax[1,1].legend(loc="lower right", frameon=True, fontsize=13.5)

In [ ]:
fig.savefig(path_img_base+'/ROC_CI_graphs'+'.pdf', format='pdf') 